I am going to use genetic algorithms to create a circuit that performs a certain unitary matrix operation on a given number of input qubits. 

Parameters :
- number of bits
- max circuit depth
- multi-qubit gates preference over single-qubit gates

Hyperparameters :
- population size
- number of generations
- mutation rate


In [1]:
import numpy as np
import pennylane as qml

In [2]:
n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

In [3]:
single_gates = [qml.Hadamard, qml.PauliX, qml.PauliY, qml.PauliZ]
multi_gates = [qml.CNOT, qml.CZ]

In [4]:
# this function generates the structure of a random circuit 
# with n_qubits with depth less than or equal to the max_depth

def gen_circuit_structure (
        n_qubits, max_depth, 
        single_gates=single_gates, multi_gates=multi_gates, single_gate_pref=0.5
        ) :
    rng = np.random.default_rng()
    depth = rng.choice(range(1, max_depth+1))

    # don't use multi gates if there is only 1 line
    if n_qubits < 2 : single_gate_pref = 1

    structure = []

    for d in range(depth) :
        # now at each depth level, assign some gates
        if rng.random() < single_gate_pref :
            wire = rng.choice(n_qubits)
            gate = rng.choice(single_gates)
            structure.append([gate, wire])
        else :
            wires = rng.choice(n_qubits, size=2, replace=False).tolist()
            gate = rng.choice(multi_gates)
            structure.append([gate, wires])

    return structure

In [46]:
@qml.qnode(dev)
def exec_circuit (structure) :
    for gate, wires in structure :
        gate(wires=wires)

    return qml.state()

In [ ]:
def create_population (pop_size) :
    population = []
    for i in range (pop_size) :
        structure = gen_circuit_structure(n_qubits, max_depth=5)
        population.append(structure)

        print(qml.draw(exec_circuit)(structure))
        U = np.array(qml.matrix(exec_circuit)(structure))

        print(U.shape)
        print()

    return population

create_population (10)

<function draw.<locals>.wrapper at 0x112a56560>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>

<function draw.<locals>.wrapper at 0x112bae200>



[[[pennylane.ops.qubit.non_parametric_ops.PauliY, 1],
  [pennylane.ops.op_math.controlled_ops.CNOT, [1, 0]],
  [pennylane.ops.qubit.non_parametric_ops.PauliX, 0]],
 [[pennylane.ops.qubit.non_parametric_ops.PauliY, 0],
  [pennylane.ops.qubit.non_parametric_ops.PauliZ, 1]],
 [[pennylane.ops.op_math.controlled_ops.CNOT, [0, 1]],
  [pennylane.ops.qubit.non_parametric_ops.PauliX, 0],
  [pennylane.ops.op_math.controlled_ops.CNOT, [1, 0]],
  [pennylane.ops.qubit.non_parametric_ops.PauliZ, 0]],
 [[pennylane.ops.op_math.controlled_ops.CNOT, [0, 1]],
  [pennylane.ops.qubit.non_parametric_ops.PauliZ, 1],
  [pennylane.ops.qubit.non_parametric_ops.Hadamard, 0],
  [pennylane.ops.op_math.controlled_ops.CNOT, [0, 1]]],
 [[pennylane.ops.op_math.controlled_ops.CZ, [1, 0]],
  [pennylane.ops.op_math.controlled_ops.CZ, [0, 1]]],
 [[pennylane.ops.qubit.non_parametric_ops.Hadamard, 0]],
 [[pennylane.ops.op_math.controlled_ops.CZ, [1, 0]]],
 [[pennylane.ops.op_math.controlled_ops.CZ, [1, 0]]],
 [[pennylane.op